In [ ]:
#pip install vaderSentiment

In [46]:
#pip install pattern


In [67]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack
import spacy
import re
import nltk
from nltk.tokenize import word_tokenize
nlp = spacy.load("en_core_web_sm")
nlp.Defaults.stop_words -= {"not", "do"}
from autocorrect import Speller
nltk.download('words')
import emoji
import sqlite3
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from nltk.corpus import words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import contractions
nltk.download('wordnet')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
from pprint import pprint

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Matthew\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Matthew\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import dataset

In [49]:
conn = sqlite3.connect("IMDB_Movies_2021.db")

cursor = conn.cursor()

table_name = "REVIEWS"

df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

conn.close()

# Drop duplicates and remove nulls

In [50]:
df = df.drop_duplicates(subset = ['REVIEW','AUTHOR'])
df = df.dropna()

# Combine review and title

In [51]:
df['FIXED_REVIEW'] = df['REVIEW'] +df['TITLE']

# Simple preprocesing: lowercase, punctuation, emojis

In [52]:
df['FIXED_REVIEW'] = [review.lower() for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'[^a-zA-Z\s]', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'@\w+', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'#\w+', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [re.sub(r'http\S+|www\S+|https\S+', '', review) for review in df['FIXED_REVIEW']]
df['FIXED_REVIEW'] = [emoji.demojize(review) for review in df['FIXED_REVIEW']]

# Remove stop short and long words, lemmatize

In [53]:
stop_words = set(stopwords.words('english'))
stop_words.remove('not')
stop_words.remove('no')
def sentiment(word):
    blob = TextBlob(word)
    sentiment = blob.sentiment.polarity
    return np.abs(sentiment)

In [54]:
def remove_stop_words(text):
    text = contractions.fix(text)
    words = text.split()    # Tokenize the text into words
    lemmatizer = WordNetLemmatizer()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    filtered_words = [word for word in filtered_words if len(word) > 2]
    filtered_words = [word for word in filtered_words if len(word) < 8]
    filtered_words = [word for word in filtered_words if sentiment(word) != 0 ]
    return ' '.join(filtered_words)

In [55]:
df['FIXED_REVIEW'] = df['FIXED_REVIEW'].apply(remove_stop_words)

# Remove misspelt words (maybe replace with better method)

In [56]:
setofwords = set(words.words())

print("hello" in setofwords)

j = 0
new_review = []

for i in df['FIXED_REVIEW']:
    words = word_tokenize(i)
    new_words = []
    for word in words:
        if word in setofwords:
            new_words.append(word)
    sentence_correct = ' '.join(new_words)        
    new_review.append(sentence_correct)
    #df['FIXED_REVIEW'][j] = sentence_correct
    j+=1


df['FIXED_REVIEW'] = new_review

True


# Add our target (ie: class)

In [57]:
def label_race(row):
    if row['RATING'] >= 7:
        return 2
    if row['RATING'] <= 3:
        return 0
    else:
        return 1

df['CLASS'] = df.apply(label_race, axis=1) 

# Add sentiment as column (can add more)

In [58]:
def get_sentiment_1(text):
    return TextBlob(text).sentiment.polarity 
df['sentiment_1'] = df['FIXED_REVIEW'].apply(get_sentiment_1)

In [59]:
analyzer = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)
    compound_score = sentiment_score['compound']
    
    return compound_score

def vader_class(text):
    sentiment_score = analyzer.polarity_scores(text)
    compound_score = sentiment_score['compound']
    
    if compound_score > 0.1:
        sentiment = "Positive"
    elif compound_score < -0.1:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"
    
    return sentiment

In [60]:
df['vader_sentiment'] = df['FIXED_REVIEW'].apply(vader_sentiment)
df['vader_class'] = df['FIXED_REVIEW'].apply(vader_class)

# Add a tokenized column for topics

In [63]:
def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

In [64]:
df['tokenized_review'] = df['FIXED_REVIEW'].apply(tokenize)

# Now use LDA for topics, gives additional context (may need to optimise how many topics

In [72]:
dictionary = corpora.Dictionary(df['tokenized_review'])
corpus = [dictionary.doc2bow(text) for text in df['tokenized_review']]

In [73]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=20, alpha="auto")

In [74]:
topic_distributions = [lda_model.get_document_topics(doc) for doc in corpus]

In [78]:
topic_matrix = []
for dist in topic_distributions:
    topic_vector = [0] * 10  # Number of topics (in this case 2)
    for topic_id, prob in dist:
        topic_vector[topic_id] = prob
    topic_matrix.append(topic_vector)

# Convert topic_matrix to a DataFrame for easy handling
topic_df = pd.DataFrame(topic_matrix, columns=[f"topic_{i}" for i in range(10)])

In [83]:
df = pd.concat([df, topic_df], axis=1)
df = df.dropna()

# Now BOW with tf idf

In [89]:
v = TfidfVectorizer(ngram_range = (1,2))
bow = v.fit_transform(df.FIXED_REVIEW.values)

In [90]:
bow = pd.DataFrame.sparse.from_spmatrix(bow, columns = v.get_feature_names_out())

In [92]:
df = pd.concat([df, bow], axis=1)


In [96]:
df = df.drop(columns = ['ID','RATING','REVIEW','TITLE'])

In [103]:
df = df.dropna()

# Save data

In [107]:
df.to_csv('preprocessed_data.csv')